# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

This is a classification problem, since the output we are going to predict is binary, which is wether or not a student will pass. While in the regression case, the prediction will often times be continious. Say predict a student's final exam score.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
#print student_data.iloc[1]
#print len(student_data.columns.values!='passed')
#print student_data.columns.values
#print len(student_data.row)
#print len(student_data[student_data['passed']=='no'])
#print student_data['school'].count()


n_students = student_data['school'].count()

# TODO: Calculate number of features
n_features = len(student_data.columns.values)-1

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed']=='yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed']=='no'])

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/float(n_students)*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

'''As one can see we have an imbalanced proportion of yes and no labels and will want to make sure the metric we 
are using for model evaluation is capturing how well the model is actually doing.
in this project we use the F1 score. As a benchmark for our final model, we can use the F1 score from predicting all
'yes' values:'''
from sklearn.metrics import f1_score
print "\nF1 score for predicting all 'yes': {:.4f}".format( 
f1_score(y_true = ['yes']*n_passed + ['no']*n_failed, y_pred = ['yes']*n_students, pos_label = 'yes', average='binary'))


Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%

F1 score for predicting all 'yes': 0.8030


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]
#print y_all
# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
#print preprocess_features(y_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [12]:
# TODO: Import any additional functionality you may need here

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

from sklearn.cross_validation import train_test_split
# TODO: Shuffle and split the dataset into the number of training and testing points above
#X_train = None
#X_test = None
#y_train = None
#y_test = None
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, stratify = y_all, test_size=95, random_state=42)

'''
Suggestion: scale the features
Feature scaling can often have a big impact on the performance of machine learning algorithms. You might
also experiment with trying to improve the F1 even more by first normalizing the features..
'''
from sklearn.preprocessing import Normalizer

normer = Normalizer()
X_train = normer.fit_transform(X_train)
X_test = normer.fit_transform(X_test)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression


- Naive Bayes
Naive Bayes learners and classifiers can be extremely fast compared to more sophisticated methods. The decoupling of the class conditional feature distributions means that each distribution can be independently estimated as a one dimensional distribution. This in turn helps to alleviate problems stemming from the curse of dimensionality.
On the flip side, although naive Bayes is known as a decent classifier, it is known to be a bad estimator, so the probability outputs from predict_proba are not to be taken too seriously.

- Decision Trees
- Some advantages of decision trees are:
- Simple to understand and to interpret. Trees can be visualised.
- Requires little data preparation. Other techniques often require data normalisation, dummy variables need to be created and blank values to be removed. Note however that this module does not support missing values.
- The cost of using the tree (i.e., predicting data) is logarithmic in the number of data points used to train the tree.
- Able to handle both numerical and categorical data. Other techniques are usually specialised in analysing datasets that have only one type of variable. See algorithms for more information.
- Able to handle multi-output problems.
- Uses a white box model. If a given situation is observable in a model, the explanation for the condition is easily explained by boolean logic. By contrast, in a black box model (e.g., in an artificial neural network), results may be more difficult to interpret.
- Possible to validate a model using statistical tests. That makes it possible to account for the reliability of the model.
- Performs well even if its assumptions are somewhat violated by the true model from which the data were generated.
- The disadvantages of decision trees include:
- Decision-tree learners can create over-complex trees that do not generalise the data well. This is called overfitting. Mechanisms such as pruning (not currently supported), setting the minimum number of samples required at a leaf node or setting the maximum depth of the tree are necessary to avoid this problem.
- Decision trees can be unstable because small variations in the data might result in a completely different tree being generated. This problem is mitigated by using decision trees within an ensemble.
- The problem of learning an optimal decision tree is known to be NP-complete under several aspects of optimality and even for simple concepts. Consequently, practical decision-tree learning algorithms are based on heuristic algorithms such as the greedy algorithm where locally optimal decisions are made at each node. Such algorithms cannot guarantee to return the globally optimal decision tree. This can be mitigated by training multiple trees in an ensemble learner, where the features and samples are randomly sampled with replacement.
- There are concepts that are hard to learn because decision trees do not express them easily, such as XOR, parity or multiplexer problems.
- Decision tree learners create biased trees if some classes dominate. It is therefore recommended to balance the dataset prior to fitting with the decision tree.

- Ensemble
The goal of ensemble methods is to combine the predictions of several base estimators built with a given learning algorithm in order to improve generalizability / robustness over a single estimator.

- bagging meta methods:
In many cases, bagging methods constitute a very simple way to improve with respect to a single model, without making it necessary to adapt the underlying base algorithm. As they provide a way to reduce overfitting, bagging methods work best with strong and complex models (e.g., fully developed decision trees), in contrast with boosting methods which usually work best with weak models (e.g., shallow decision trees).

- Nearest Neighbors
Supervised neighbors-based learning comes in two flavors: classification for data with discrete labels, and regression for data with continuous labels.Neighbors-based methods are known as non-generalizing machine learning methods, since they simply “remember” all of its training data. Despite its simplicity, nearest neighbors has been successful in a large number of classification and regression problems, including handwritten digits or satellite image scenes. Being a non-parametric method, it is often successful in classification situations where the decision boundary is very irregular.

- Stochastic Gradient Descent (SGDC)
SGD has been successfully applied to large-scale and sparse machine learning problems often encountered in text classification and natural language processing. Given that the data is sparse, the classifiers in this module easily scale to problems with more than 10^5 training examples and more than 10^5 features.
- The advantages of Stochastic Gradient Descent are:
Efficiency.
Ease of implementation (lots of opportunities for code tuning).
- The disadvantages of Stochastic Gradient Descent include:
SGD requires a number of hyperparameters such as the regularization parameter and the number of iterations.
SGD is sensitive to feature scaling.
The major advantage of SGD is its efficiency, which is basically linear in the number of training examples. If X is a matrix of size (n, p) training has a cost of O(k n \bar p), where k is the number of iterations (epochs) and \bar p is the average number of non-zero attributes per sample.

- SVM
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

- The advantages of support vector machines are:
Effective in high dimensional spaces.
Still effective in cases where number of dimensions is greater than the number of samples.
Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.
Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

- The disadvantages of support vector machines include:
If the number of features is much greater than the number of samples, the method is likely to give poor performances.
SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).

- Logistic Regression
Logistic regression, despite its name, is a linear model for classification rather than regression. Logistic regression is also known in the literature as logit regression, maximum-entropy classification (MaxEnt) or the log-linear classifier. In this model, the probabilities describing the possible outcomes of a single trial are modeled using a logistic function.

- Real world application 


- Spam detection: Bayesian classification, k-NN, ANNs, SVMs (google)
- Fraud detection:un-/supervised
Supervised neural networks, fuzzy neural nets, and combinations of neural nets and rules, have been extensively explored and used for detecting fraud in mobile phone networks and financial statement fraud. (wikipedia)
Bayesian learning neural network is implemented for credit card fraud detection, telecommunications fraud, auto claim fraud detection, and medical insurance fraud(wikipedia)
Some important studies with unsupervised learning with respect to fraud detection should be mentioned. For example, Bolton and Hand[11] use Peer Group Analysis and Break Point Analysis applied on spending behaviour in credit card accounts. Peer Group Analysis detects individual objects that begin to behave in a way different from objects to which they had previously been similar. Another tool Bolton and Hand[11] develop for behavioural fraud detection is Break Point Analysis. Unlike Peer Group Analysis, Break Point Analysis operates on the account level. A break point is an observation where anomalous behaviour for a particular account is detected. Both the tools are applied on spending behaviour in credit card accounts.(wikipedia)

- Handwritten digit recognition: linear classifier, knn, ann, svm, principal component analysis, polynomial classifier

- Speech recognition: deep feedforward and recurrent neural networks (wiki)
https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/tasl-deng-2244083-x_2.pdf

- Face detection:
https://arxiv.org/abs/1503.03832

- Product recommendation:
https://www.quora.com/How-exactly-is-machine-learning-used-in-recommendation-engines



### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

1. Bagged Decission Tree
    1.1 General App: Given remote sensing data from satelite, classify different types of land: mountainous terrain, urban terrain, agricultural lands or Wetlands.
    1.2 Strengths and Weaknesses
        Pros:
            reduces variance in comparison to regular decision trees
            Can provide variable importance measures
                classification: Gini index
                regression: RSS
            Can easily handle qualitative (categorical) features
            Out of bag (OOB) estimates can be used for model validation
        Cons:
            Not as easy to visually interpret
            Does not reduce variance if the features are correlated
    1.3 why this model: This model could deal with classification problem with small data samples. The variance reducing ability might be especially useful in this case. Bagged tree can easily handle qualitative and categorical features. The given data has both.

2. KNN
    2.1 General App: classifying data into clusters in unsupervised learning; recommend a similar product.
    2.2 Strengths and Weaknesses
        Pros:
            simple, cheap, no training involved("lazy")
            naturally handle multiclass classification and regression
        Cons:
            expensive and slow predicting new instances
            distance function must be difined in a meaningful way to data
            perform poorly on high dimension dataset
    2.3 why this model: This model is a natural pick for classification problem. As mentioned in the sklearn "Choosing the right estimator"(http://scikit-learn.org/stable/tutorial/machine_learning_map/), it is the first picked algorithm. We don't want to pick a complex model, while the problem could be solved be a simpler one. As we shall not assume the data given is complex or not in the first place.

3. SVC
    3.1 General App: content based image retrival; facial expression classification; text classification; protein structure prediction.
    3.2 Strengths and Weaknesses
        Pros:
            Performs similarly to logistic regression when linear separation
            Performs well with non-linear boundary depending on the kernel used
            Handle high dimensional data well
            Robust to noise (because they maximize margins)
        Cons:
            Susceptible to overfitting/training issues depending on kernel
            Need to select a good kernel function
            Model parameters are difficult to interpret
            Sometimes numerical stability problems
            Requires significant memory and processing power
     3.3 why this model: This model also deals with classification problem. It is chosen because it is more flexible. There are more parameterrs to play with. While in our case, we have a small data sample, it won't cost too much memory and processing power.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [13]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    start = time()
    #from sklearn.cross_validation import ShuffleSplit
    #cv = ShuffleSplit(X_train.shape[0], n_iter=10, test_size=0.2, random_state=0)
    
    #from sklearn.grid_search import GridSearchCV
    #gammas = np.logspace(-6, -1, 10)
    #classifier = GridSearchCV(estimator=clf, cv=cv, param_grid=dict(gamma=gammas))
    #classifier.fit(X_train, y_train)
    # Start the clock, train the classifier, then stop the clock
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    from sklearn.metrics import fbeta_score
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    #f1_score = make_scorer(fbeta_score, beta=2)
    return fbeta_score(target.values, y_pred, beta=2, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [14]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from skearln import model_C
from sklearn.metrics import make_scorer
from sklearn.metrics import fbeta_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

#from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors.nearest_centroid import NearestCentroid

from sklearn.ensemble import RandomForestClassifier

from sklearn.svm import SVC

# TODO: Initialize the three models
clf_A = BaggingClassifier(DecisionTreeClassifier(random_state=42))
clf_B = NearestCentroid()
clf_C = SVC(random_state=42)

# TODO: Set up the training set sizes
X_train_100 = X_train[:99]
y_train_100 = y_train[:99]

X_train_200 = X_train[:199]
y_train_200 = y_train[:199]

X_train_300 = X_train[:299]
y_train_300 = y_train[:299]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
#print y_train_100
#bagged tree
#train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
#train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
#train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

#KNN
#train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
#train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
#train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

#Random Forest
#train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
#train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
#train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

#SVM
#train_predict(clf_D, X_train_100, y_train_100, X_test, y_test)
#train_predict(clf_D, X_train_200, y_train_200, X_test, y_test)
#train_predict(clf_D, X_train_300, y_train_300, X_test, y_test)

for clf in [clf_A, clf_B, clf_C]:
    for size in [100, 200, 300]:
        train_predict(clf, X_train[:size], y_train[:size], X_test, y_test)

'''Suggestion: look at confusion matrix
To get a closer look at how the models are predicting the labels, you can also experiment with plotting a confusion
matrix...
'''

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

#compute the confusion matrix for a model
model = clf_C
cm = confusion_matrix(y_test.values, model.predict(X_test))

#view with a heatmap
sns.heatmap(cm, annot=True, cmap='Blues', xticklabels=['no','yes'], yticklabels=['no', 'yes'])
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Confusion matrix for:\n{}'.format(model.__class__.__name__));

Training a BaggingClassifier using a training set size of 100. . .
Trained model in 0.0649 seconds
Made predictions in 0.0012 seconds.
F1 score for training set: 0.9756.
Made predictions in 0.0014 seconds.
F1 score for test set: 0.6349.
Training a BaggingClassifier using a training set size of 200. . .
Trained model in 0.0417 seconds
Made predictions in 0.0013 seconds.
F1 score for training set: 0.9737.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.6329.
Training a BaggingClassifier using a training set size of 300. . .
Trained model in 0.0548 seconds
Made predictions in 0.0017 seconds.
F1 score for training set: 0.9681.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7407.
Training a NearestCentroid using a training set size of 100. . .
Trained model in 0.0003 seconds
Made predictions in 0.0081 seconds.
F1 score for training set: 0.6013.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.6070.
Training a NearestCentroid using a training set size o

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Bagged Tree**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0526          |          0.0012        |       0.9925        |    0.6769       |
| 200               |        0.0320           |      0.0012            |       0.9847      |    0.7132       |
| 300               |        0.0344           |       0.0022           |     0.9925       |    0.7481       |

** Classifer 2 - Nearest centoid**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0009          |     0.0004             |        0.5556       |     0.5310      |
| 200               |     0.0009              |      0.0006            |       0.6868        |     0.6880      |
| 300               |           0.0009        |        0.0004          |      0.7018         |     0.6667      |
 
** Classifer 3 - SVM **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0019             |          0.0010        |     0.8354       |       0.8025      |
| 200               |         0.0038          |          0.0013        |     0.8431       |       0.8105      |
| 300               |          0.0077         |          0.0014        |     0.8664       |       0.8052      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

The best model is SVC. SVC is known to solve classification problems. In our case is classify students as 'passed' or 'fail'. There are around 400 samples in our data. The model uses a subset of that data in decision function, so it further reduces memory consumption. In terms of training time, it is considerably fast. The model shows a test score of 0.787, which is among the best.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

The final model chosen is Support Vector Classifier. Let's consider a simple problem: for a 2D set of points, find a straight seperating line. SVM define the worth of line as: A line is bad if it passes too close to the points because it will be noise sensitive and it will not generalize correctly. Therefore, our goal should be to find the line passing as far as possible from all points. Thus the algorithm is based on finding the largest minimun distance to the training samples. This distance is called Margin within SVM algorithm.
![Image of SVM](https://udacity-github-sync-content.s3.amazonaws.com/_imgs/372/1473447655/SVM_2.png)
As showed in the picture, the dotted line is the optimal seperating straight line. The minimun distance between the solid lines is margin. The closest data samples are called support vector.

SVM works as well when dataset is not linear seperatable. Consider the following dataset:
![Image of nonlinear dataset](http://www.eric-kim.net/eric-kim-net/posts/1/imgs/dataset_nonsep.png)

There is no linear line here seperates the data well.
The way SVM deals with this problem is called kernel trick. What kernel trick does is it takes the input and maps them into higher dimensions. Looking at the left image below, the kernel trick maps the original dataset to a 3D space. Now the dataset is seperatable. The decision boundary is depicted as a green hyperplane.
![Image of nonlinear dataset in R3](http://www.eric-kim.net/eric-kim-net/posts/1/imgs/data_2d_to_3d_hyperplane.png)

When we map our dataset and decision boundary back to 2D space, we get the picture on the right.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [15]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.svm import SVC
from sklearn.cross_validation import ShuffleSplit, StratifiedShuffleSplit
from sklearn.metrics import f1_score



# TODO: Create the parameters list you wish to tune
parameters = {'kernel': ['rbf', 'poly', 'sigmoid'], 'degree': [2, 3, 4], 'gamma': [1e-30, 1e-40, 1e-50, 1e-60],
                     'C': [1, 10, 100, 1000]}

ssscv = StratifiedShuffleSplit(y_train, n_iter=10, test_size=0.1)
#cv = ShuffleSplit(X_train.shape[0], n_iter=10, test_size=0.2, random_state=0)

# TODO: Initialize the classifier
clf = SVC()

'''
Suggestion: using F-beta
The F1 score assigns equal weighting to precision and recall, but in some cases you might want to optimize for one over
the other (eg, over-weighting recall to detect more of the 'not passing' students). For that you can use F-beta score
instead...
beta < 1: more weight to precision
beta > 1: more weight to recall
'''
# TODO: Make an f1 scoring function using 'make_scorer' 
f2_scorer = make_scorer(fbeta_score, beta=2, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=clf, param_grid=parameters, scoring=f2_scorer, cv=ssscv)

# TODO: Fit the grid search object to the training data and find the optimal parameters
gird_obj=grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0035 seconds.
Tuned model has a training F1 score of 0.9103.
Made predictions in 0.0012 seconds.
Tuned model has a testing F1 score of 0.9117.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

.8050 is the final score. It does not show much improvements.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.